# **HW1: Regression** 
In *assignment 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part

# 1. Basic Part (60%)
In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

## Global attributes
Define the global attributes

In [ ]:
input_dataroot = 'hw1_basic_input.csv' # Input file named as 'hw1_basic_input.csv'
output_dataroot = 'hw1_basic.csv' # Output file will be named as 'hw1_basic.csv'

input_datalist =  [] # Initial datalist, saved as numpy array
output_datalist =  [] # Your prediction, should be 10 * 4 matrix and saved as numpy array
             # The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

You can add your own global attributes here


In [ ]:
random.seed(1)

## Load the Input File
First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in *input_datalist*

In [ ]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
    input_datalist = np.array(list(csv.reader(csvfile)))

## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 1: Split Data
Split data in *input_datalist* into training dataset and validation dataset 



In [ ]:
# GenerateInput(add Case(t-1))
def GenerateInput(dataset):
    col = np.roll(dataset[:,1], 1) # col = case number of preWeek
    col = np.array(list(zip(col)))
    return np.append(col, dataset, axis=1)

# SplitData
def SplitData(dataset):
    train_dataset, test_dataset = dataset[47:], dataset[85:] # Split
    return (train_dataset, test_dataset)

### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [ ]:
# PreprocessData
def PreprocessData(dataset, Q):
    Q1, Q2, Q3 = Q
    # MissingData(sol: Filled with Q2)
    dataset = np.array([[Q2, r[0], r[2]] if '' in r else [r[1], r[0], r[2]] for r in dataset]).astype(np.float64)
    # Outlier(sol: Drop)
    bound = 1.5 * (Q3 - Q1) # bound = +- 1.5*IQR
    dataset = np.array([i for i in dataset if i[0] > Q1-bound and i[0] < Q3+bound])
    return dataset

### Step 3: Implement Regression
> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part




In [ ]:
# Regression(Gradient Descent)
def Regression(X, y):
    #initial
    learning_rate = 0.0001
    n = 10000
    theta = random.random()
    weight = np.random.rand(2)
    min_cost = math.inf
    flag = 0
    
    for iteration in range(n):
        y_hat = theta + np.dot(X, weight)
        y_res = y_hat - y
        cost = np.sum(np.dot(y_res.T, y_res)) / (len(y)*len(y))
        if cost < min_cost: ++flag
        if not flag%3 : learning_rate *= 0.9
        min_cost = min(cost, min_cost)
        dt = np.sum(y_res) / len(y)
        dw = np.dot(y_res, X) / len(y)
        theta -= learning_rate*2*dt
        weight -= learning_rate*dw
        
    return (theta, weight)

# Evaluation
def Eval(Y, Y_pre):
    return np.sum(np.absolute(Y-Y_pre)/Y)/len(Y)*100

### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [ ]:
# MakePrediction
def MakePrediction(dataset, case_prev, coef):
    prediction = []
    for i in dataset:
        cur = coef[0] + i*coef[1] + case_prev*coef[2]
        prediction.append(cur)
        pre = cur
    return prediction

### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be: 
```
3 2 1
```





In [ ]:
def Model(dataset, Q):
    train_dataset, test_dataset = SplitData(dataset)
    train_dataset, test_dataset = PreprocessData(train_dataset, Q), PreprocessData(test_dataset, Q)
    X_train, y_train = train_dataset[:,:2], train_dataset[:,2]
    X_test, y_test = test_dataset[:,:2], test_dataset[:,2]
    min_eval = math.inf
    for t in range(100):
        theta0, weight0 = Regression(X_train, y_train)
        y_pre = theta0 + np.dot(X_test, weight0)
        val = Eval(y_test, y_pre)
        if val < min_eval:
            min_eval = val
            theta, weight = theta0, weight0
    return [*weight.tolist(), theta]

# coef = [eval, weight[0](Temperature), weight[1](CaseofPrevWeek), theta]
coef = [[]]*3
input_data = [input_datalist[1:95,i::3] for i in range(1, 4)]
output_data = [input_datalist[95:,i].astype(np.float64) for i in range(1, 4)]

for i, data in enumerate(input_data):
    dataset = GenerateInput(data)
    valid_dataset = np.array([row for row in dataset if '' not in row]).astype(np.float64)
    Q = [np.percentile(valid_dataset[:,1], 25*i) for i in range(1,4)]
    coef[i] = Model(dataset, Q)

# case_predict = coef_0*Temperature + coef_1*CaseofPrevWeek + coef_2
for i, row in enumerate(coef):
    print(chr(65+i), *row)

A 0.4230755630621288 0.6054281404463512 0.11606595999479018
B 0.7686972068305014 0.21647129531893558 0.24849235608027498
C 0.6105556184517776 0.514931820647193 0.6892163871029104


## Write the Output File
Write the prediction to output csv
> Format: 'epiweek', 'CityA', 'CityB', 'CityC'

In [ ]:
prev = list(data[-1,1] for data in input_data)
output_datalist = [[i for i in range(202143, 202153)]]

for i, data in enumerate(output_data):
    output_datalist.append(MakePrediction(data, float(prev[i]), coef[i]))
    
output_datalist = list(zip(*output_datalist))
with open(output_dataroot, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_datalist)

# 2. Advanced Part (35%)
In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv** 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import csv
pd.options.mode.chained_assignment = None

data = pd.read_csv('hw1_basic_input.csv')
data = data.drop(columns=['epiweek'])
output = 'hw1_advanced.csv'

past=[74, 74, 84]
output_datalist = [[i for i in range(202143, 202153)],[],[],[]]

def Model(i, df):
    # SplitData
    X = df.drop(columns=['C']).values[:94]
    y = df['C'].values[:94]
    X_ans = df['C'].values[90:94].tolist()
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train, X_test = X[past[i]:], X[89:]
    y_train, y_test = y[past[i]:], y[89:]
    
    # Regression
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge, Lasso
    
    line_reg = Lasso()
    line_reg.fit(X_train, y_train)
    
    # MakePridiction
    for j in range(10):
        val = sum(X_ans)/len(X_ans)*line_reg.coef_[0]+line_reg.intercept_
        output_datalist[i+1].append(val)
        X_ans.pop(0)
        X_ans.append(val)
        
    # Evaluation
    from sklearn.metrics import mean_absolute_percentage_error as mape
    #print(mape(y_train, line_reg.predict(X_train)), mape(y_test, line_reg.predict(X_test)))
    
    return line_reg.coef_[0], line_reg.intercept_
for i in range(3):
    df = data[data.columns[[i+3]]]
    df.columns = ['C']
    df['Average'] = df['C'].rolling(4, closed='left').mean()
    prediction = Model(i, df)
    print(chr(65+i), *prediction)
    
#print(output_datalist)
output_datalist = list(zip(*output_datalist))
with open(output, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_datalist)

A 0.8438837122923794 2.6146953957716264
B -0.15528106881774636 26.52829594151752
C 0.46999353882406597 18.82729656797537


# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered 
*   Summarize your work and your reflections 
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)

In [1]:
learning_rate = 0.1
weight = [1,1]
X = [2,1]
y_hat = 3
y_res = 2
dw = np.dot(y_res, X)
dw

NameError: ignored